# Lambda = 4, lambda = 1.0: EvolvedOperatorAnsatz

In [1]:
import numpy as np
import pandas as pd
import pylab
import matplotlib.pyplot as plt
import time

import qiskit
from qiskit.quantum_info import Pauli
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import NumPyEigensolver
from qiskit.circuit.library import EvolvedOperatorAnsatz

import sys
sys.path.append('../utility')
from vqe_run import *
from qc_ansatze import *

In [2]:
def open_n_process(file_link):
    file = open(file_link, "r")
    content = file.read()

    data = content.split(',\n')
    data[0] = data[0].split('\n')[1]
    data[-1] = data[-1].split('\n')[0]

    data_coeff = []
    data_str = []
    for i in range(len(data)):
        data_coeff.append(float(data[i].split('*')[0]))
        data_str.append(data[i].split('*')[1].split(' ')[1])
    return data_coeff, data_str
    
def check_min_max(data_coeff):
    print(f'Min absolute value is {np.round(np.min(np.abs(data_coeff[1:])),6)}')
    print(f'Max absolute value is {np.round(np.max(np.abs(data_coeff[1:])),5)}')
    print(f'Mean absolute value is {np.round(np.mean(np.abs(data_coeff[1:])),5)}')
    
def get_k_largest_ops(data_coeff, data_str, k):
    ind_list = np.argsort(np.abs(data_coeff[1:]))[-k:]
    ops = [data_str[1:][i] for i in ind_list]
    val= [np.round(data_coeff[1:][i],4) for i in ind_list]
    return ind_list, ops, val

# Load H

In [3]:
data_coeff, data_str = open_n_process("../utility/pauliH_L4_g1.0.txt")
Hpauli = list(zip(data_str, data_coeff))
H4q = SparsePauliOp.from_list(Hpauli)
solver = NumPyEigensolver(k=4)
exact_solution = solver.compute_eigenvalues(H4q)
#print("Exact Result of qubit hamiltonian:", np.real(exact_solution.eigenvalues))
E_exact = np.round(np.real(exact_solution.eigenvalues)[0],5)
print(f'{check_min_max(data_coeff)}')
print(f'E_exact = {E_exact}')

Min absolute value is 0.004487
Max absolute value is 1.75
Mean absolute value is 0.13128
None
E_exact = 3.52625


In [4]:
#get the indices of the N largest elements
ind15, ops15, vals15 = get_k_largest_ops(data_coeff, data_str, 15)
ind20, ops20, vals20 = get_k_largest_ops(data_coeff, data_str, 20)
ind25, ops25, vals25 = get_k_largest_ops(data_coeff, data_str, 25)
ind30, ops30, vals30 = get_k_largest_ops(data_coeff, data_str, 30)
ind40, ops40, vals40 = get_k_largest_ops(data_coeff, data_str, 40)

ops_Hp15 =[Pauli(f'{ops15[i]}') for i in range(len(ops15))]
ops_Hp20 =[Pauli(f'{ops20[i]}') for i in range(len(ops20))]
ops_Hp25 =[Pauli(f'{ops25[i]}') for i in range(len(ops25))]
ops_Hp30 =[Pauli(f'{ops30[i]}') for i in range(len(ops30))]
ops_Hp40 =[Pauli(f'{ops40[i]}') for i in range(len(ops40))]

In [12]:
#[print(list(zip(ops30, vals30))[i]) for i in range(30)]

In [5]:
ev_op_Hp15 = EvolvedOperatorAnsatz(ops_Hp15, reps=1, insert_barriers=True)
ev_op_Hp20 = EvolvedOperatorAnsatz(ops_Hp20, reps=1, insert_barriers=True)
ev_op_Hp25 = EvolvedOperatorAnsatz(ops_Hp25, reps=1, insert_barriers=True)
ev_op_Hp30 = EvolvedOperatorAnsatz(ops_Hp30, reps=1, insert_barriers=True)
ev_op_Hp40 = EvolvedOperatorAnsatz(ops_Hp40, reps=1, insert_barriers=True)

ev_op_Hp15_2f = EvolvedOperatorAnsatz(ops_Hp15, reps=2, insert_barriers=True)
ev_op_Hp20_2f = EvolvedOperatorAnsatz(ops_Hp20, reps=2, insert_barriers=True)
ev_op_Hp25_2f = EvolvedOperatorAnsatz(ops_Hp25, reps=2, insert_barriers=True)
ev_op_Hp30_2f = EvolvedOperatorAnsatz(ops_Hp30, reps=2, insert_barriers=True)
ev_op_Hp40_2f = EvolvedOperatorAnsatz(ops_Hp40, reps=2, insert_barriers=True)

ansatz_list = [ev_op_Hp15, ev_op_Hp20, ev_op_Hp25, ev_op_Hp30, ev_op_Hp40, 
               ev_op_Hp15_2f, ev_op_Hp20_2f, ev_op_Hp25_2f, ev_op_Hp30_2f, ev_op_Hp40_2f]

ansatz_names = ['ev_op_Hp15', 'ev_op_Hp20', 'ev_op_Hp25', 'ev_op_Hp30', 'ev_op_H40',
               'ev_op_Hp15_2f','ev_op_Hp20_2f','ev_op_Hp25_2f', 'ev_op_Hp30_2f', 'ev_op_Hp40_2f']

print(f'number of params: {[ansatz_list[i].num_parameters for i in range(len(ansatz_list))]}')

number of params: [15, 20, 25, 30, 40, 30, 40, 50, 60, 80]


In [5]:
#additional ansatze
ev_op_Hp15_3f = EvolvedOperatorAnsatz(ops_Hp15, reps=3, insert_barriers=True)
ev_op_Hp20_3f = EvolvedOperatorAnsatz(ops_Hp20, reps=3, insert_barriers=True)
ansatz_list = [ev_op_Hp15_3f, ev_op_Hp20_3f]
ansatz_names =['ev_op_Hp15_3f', 'ev_op_Hp20_3f']
print(f'number of params: {[ansatz_list[i].num_parameters for i in range(len(ansatz_list))]}')

number of params: [45, 60]


# QVE individual runs

In [6]:
seed = 170
iterations = 650
algorithm_globals.random_seed = seed

#estimator
noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)
#storing values
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
    
def run_qve_w_specified_optimizer(optimizer, ansatz):
    opt = optimizer(maxiter = iterations)
    vqe = VQE(noiseless_estimator, ansatz, optimizer=opt, callback=store_intermediate_result)
    result = vqe.compute_minimum_eigenvalue(operator=H4q).eigenvalue.real
    print(f"VQE result: {result:.5f}")
    return result

# COBYLA, 

### 550 iterations

At step 0, with ev_op_Hp20
VQE result: 3.48950
Length of this optimization 273, time taken = 392.983 

At step 1, with ev_op_Hp30
VQE result: 4.08074
Length of this optimization 353, time taken = 572.079 

At step 2, with ev_op_H40
VQE result: 4.13562
Length of this optimization 528, time taken = 1315.623 

At step 3, with ev_op_Hp20_2f
VQE result: 3.57098
Length of this optimization 550, time taken = 1164.852 

At step 4, with ev_op_Hp30_2f
VQE result: 4.01289
Length of this optimization 550, time taken = 1298.156 

At step 5, with ev_op_H40_2f
VQE result: 5.65374
Length of this optimization 550, time taken = 2171.84 

In [7]:
r_cobyla=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')    
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(COBYLA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_cobyla.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))
    

At step 0, with ev_op_Hp15
VQE result: 3.62082
Length of this optimization 187, time taken = 222.408 

At step 1, with ev_op_Hp20
VQE result: 3.84073
Length of this optimization 237, time taken = 272.391 

At step 2, with ev_op_Hp25
VQE result: 3.35342
Length of this optimization 326, time taken = 468.618 

At step 3, with ev_op_Hp30
VQE result: 3.36268
Length of this optimization 454, time taken = 676.014 

At step 4, with ev_op_H40
VQE result: 4.34683
Length of this optimization 538, time taken = 1160.16 

At step 5, with ev_op_Hp15_2f
VQE result: 4.35314
Length of this optimization 305, time taken = 511.484 

At step 6, with ev_op_Hp20_2f
VQE result: 3.75998
Length of this optimization 467, time taken = 852.588 

At step 7, with ev_op_Hp25_2f
VQE result: 4.92616
Length of this optimization 527, time taken = 1056.118 

At step 8, with ev_op_Hp30_2f
VQE result: 4.71595
Length of this optimization 650, time taken = 1419.445 

At step 9, with ev_op_Hp40_2f
VQE result: 12.45933
Length of

In [9]:
df1 = pd.concat([r_cobyla[i] for i in range(len(r_cobyla))], axis = 1)
df1.to_csv('l4_l10_op_ev_20_30_40_indv_cobyla.csv')

In [7]:
# Additional ansatze
r_cobyla2=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(COBYLA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_cobyla2.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))

At step 0, with ev_op_Hp15_3f
VQE result: 4.64766
Length of this optimization 463, time taken = 1256.847 

At step 1, with ev_op_Hp20_3f
VQE result: 3.67295
Length of this optimization 650, time taken = 2065.229 



In [8]:
df1b = pd.concat([r_cobyla2[i] for i in range(len(r_cobyla2))], axis = 1)
df1b.to_csv('l4_l10_op_ev_a_20_30_40_indv_cobyla.csv')
df1b

,ev_op_Hp15_3f,ev_op_Hp20_3f
0,17.222337,22.514781
1,14.190180,22.514781
2,16.957282,22.514781
3,15.933015,22.514781
4,16.074259,22.514781
...,...,...
645,NaN,3.755489
646,NaN,3.689009
647,NaN,3.722354
648,NaN,3.727650


# SPSA,

### 550 iterations

At step 0, with ev_op_Hp20
VQE result: 3.47956
Length of this optimization 1151, time taken = 2054.015 

At step 1, with ev_op_Hp30
VQE result: 3.52841
Length of this optimization 1151, time taken = 2210.211 

At step 2, with ev_op_H40
VQE result: 3.67711
Length of this optimization 1151, time taken = 2918.526 

At step 3, with ev_op_Hp20_2f
VQE result: 3.55648
Length of this optimization 1151, time taken = 2482.162 

At step 4, with ev_op_Hp30_2f
VQE result: 3.58351
Length of this optimization 1151, time taken = 2808.639 

At step 5, with ev_op_H40_2f
VQE result: 7.49683
Length of this optimization 1151, time taken = 3929.957 



In [8]:
r_spsa=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')    
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(SPSA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_spsa.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))

At step 0, with ev_op_Hp15
VQE result: 3.75863
Length of this optimization 1351, time taken = 1796.695 

At step 1, with ev_op_Hp20
VQE result: 3.50884
Length of this optimization 1351, time taken = 1840.615 

At step 2, with ev_op_Hp25
VQE result: 3.48718
Length of this optimization 1351, time taken = 2068.862 

At step 3, with ev_op_Hp30
VQE result: 3.54551
Length of this optimization 1351, time taken = 20877.993 

At step 4, with ev_op_H40
VQE result: 3.59112
Length of this optimization 1351, time taken = 7329.858 

At step 5, with ev_op_Hp15_2f
VQE result: 3.71765
Length of this optimization 1351, time taken = 2245.91 

At step 6, with ev_op_Hp20_2f
VQE result: 3.45539
Length of this optimization 1351, time taken = 2385.868 

At step 7, with ev_op_Hp25_2f
VQE result: 3.76259
Length of this optimization 1351, time taken = 2723.849 

At step 8, with ev_op_Hp30_2f
VQE result: 3.71206
Length of this optimization 1351, time taken = 3034.612 

At step 9, with ev_op_Hp40_2f
VQE result: 9.

In [12]:
df2 = pd.concat([r_spsa[i] for i in range(len(r_spsa))], axis = 1)
df2.to_csv('l4_l10_op_ev_20_30_40_indv_spsa.csv')

In [9]:
r_spsa2=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(SPSA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_spsa2.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))

At step 0, with ev_op_Hp15_3f
VQE result: 3.67341
Length of this optimization 1351, time taken = 3932.001 

At step 1, with ev_op_Hp20_3f
VQE result: 3.49233
Length of this optimization 1351, time taken = 28971.853 



In [10]:
df2b = pd.concat([r_spsa2[i] for i in range(len(r_spsa2))], axis = 1)
df2b.to_csv('l4_l10_op_ev_a_20_30_40_indv_spsa.csv')